In [241]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

In [242]:
def clean_names(names):
    return names.str.lower().str.replace(' ', '').str.replace('-','')

## Top 100 mineable coins

In [243]:
# Top 100 mineable coins
url = 'https://coinmarketcap.com/coins/views/filter-non-mineable/'

r = requests.get(url) 

page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
table = soup.find_all('div', class_='cmc-table__table-wrapper-outer')

In [247]:
# Get column names
column_names = []
for header in table[0].find_all('th'):
    column_names.append(str(header.string))
# Drop graphs
column_names.pop()
column_names.pop()

data = []
for row in table[2].find_all('tr')[1:]:
    new_row = []
    for entry in row:
        new_row.append(entry.text)
    new_row.pop()
    new_row.pop()
    data.append(new_row)
mineable_100 = pd.DataFrame(data, columns = column_names)
mineable_100 = df.set_index('Name')
mineable_100 = mineable_100.set_index(clean_names(mineable_100.index))
mineable_100 = mineable_100[['#', "Market Cap", "Price"]]

## What to mine, nethash

### ASIC

In [248]:
url = 'https://whattomine.com/asic.json'

r = requests.get(url) 
data = r.json()

whattomine_asic = pd.DataFrame(data)
# normalize + clean names
whattomine_asic = pd.json_normalize(whattomine_asic['coins']).set_index(clean_names(whattomine_data.index.str.lower()))
whattomine_asic['block_time'] = pd.to_numeric(whattomine_asic['block_time'])
whattomine_asic['nethash'] = pd.to_numeric(whattomine_asic['nethash'])
# Drop some columns
whattomine_asic = whattomine_asic[['tag','algorithm', 'block_time', 'difficulty', 'nethash']]
whattomine_asic[:8]

,tag,algorithm,block_time,difficulty,nethash
actinium,ACM,Lyra2z,147.0,6.498831e+03,1.898794e+11
axe,AXE,X11,157.0,9.043869e+05,2.474084e+13
bitcoin,BTC,SHA-256,673.0,1.655292e+13,1.056378e+20
bitcoincashabc,BCH,SHA-256,600.0,5.434658e+11,3.890279e+18
bitcoindiamond,BCD,BCD,644.0,2.943596e+06,1.963144e+13
bitcoinsv,BSV,SHA-256,571.0,3.741698e+11,2.814443e+18
bytecoin,BCN,CryptoNight,116.0,2.225656e+10,1.918669e+08
dgbqubit,DGB,Qubit,75.0,2.140807e+06,1.225959e+14


### GPU

In [249]:
url = 'https://whattomine.com/coins.json'

r = requests.get(url) 
data = r.json()

whattomine_gpu = pd.DataFrame(data)
whattomine_gpu = pd.json_normalize(whattomine_gpu['coins']).set_index(clean_names(whattomine_gpu.index.str.lower()))
whattomine_gpu['block_time'] = pd.to_numeric(whattomine_gpu['block_time'])
whattomine_gpu['nethash'] = pd.to_numeric(whattomine_gpu['nethash'])
# Drop some columns
whattomine_gpu = whattomine_gpu[['tag','algorithm', 'block_time', 'difficulty', 'nethash']]
whattomine_gpu[:4]

,tag,algorithm,block_time,difficulty,nethash
aeternity,AE,CuckooCycle,185.0,2.688685e+07,145334
aion,AION,"Equihash (210,9)",11.0,1.382933e+07,1257211
anon,ANON,EquihashZero,157.0,3.574410e+05,2276
beam,BEAM,BeamHashII,60.0,1.153832e+08,1923053


## CoinWarz

In [250]:
url = 'https://www.coinwarz.com/v1/api/profitability?apikey=58467a6f2e2d4c65b7ef720b26137453&algo=all'
r = requests.get(url) 
data = r.json()

coinwarz = pd.DataFrame(data)
    
coinwarz = pd.json_normalize(coinwarz['Data'])
coinwarz = coinwarz.set_index(clean_names(coinwarz['CoinName']))

# Drop and rename some columns
coinwarz = coinwarz[['CoinTag', 'Algorithm','BlockTimeInSeconds', 'Difficulty']]
coinwarz = coinwarz.rename(columns={'CoinTag': 'tag', 'Algorithm': 'algorithm','BlockTimeInSeconds':'block_time', 'Difficulty': 'difficulty'})
coinwarz['nethash_scraped'] = 0
coinwarz

,tag,algorithm,block_time,difficulty,nethash_scraped
CoinName,,,,,
gamecredits,GAME,Scrypt,90,1.302676e+03,0
ethereumclassic,ETC,EtHash,15,1.604594e+14,0
ethereum,ETH,EtHash,15,2.153148e+15,0
verge,XVG,Scrypt,30,8.451237e+03,0
cannabiscoin,CANN,X11,42,2.657983e+04,0
zcash,ZEC,Equihash,75,3.940947e+07,0
horizen,ZEN,Equihash,150,1.437516e+07,0
peercoin,PPC,SHA-256,600,1.550367e+09,0
feathercoin,FTC,NeoScrypt,60,9.628761e+01,0


# Combining the data

In [251]:
def check_missing(data):
    missing_data = result[data.isnull().any(axis=1)]
    return missing_data.shape[0]

In [252]:
whattomine = whattomine_gpu.append(whattomine_asic)
whattomine = whattomine.rename(columns={'nethash':'nethash_scraped'})
whattomine

,tag,algorithm,block_time,difficulty,nethash_scraped
aeternity,AE,CuckooCycle,185.0,2.688685e+07,1.453340e+05
aion,AION,"Equihash (210,9)",11.0,1.382933e+07,1.257211e+06
anon,ANON,EquihashZero,157.0,3.574410e+05,2.276000e+03
beam,BEAM,BeamHashII,60.0,1.153832e+08,1.923053e+06
bitcash,BITC,X25X,63.0,3.600000e+01,2.454267e+09
...,...,...,...,...,...
vergegroestl,XVG,Myr-Groestl,150.0,5.903233e+06,1.690280e+14
vergelyra2rev2,XVG,Lyra2REv2,150.0,3.220991e+04,9.222701e+11
vergescrypt,XVG,Scrypt,150.0,6.986193e+03,2.000365e+11
viacoin,VIA,Scrypt,23.0,3.010433e+05,5.621614e+13


In [253]:
result = mineable_100.join(whattomine, how="left")

In [254]:
for coin in coinwarz.index:
    if coin not in whattomine.index.values:
        if coin in result.index:
            result.loc[coin, ['tag','algorithm','block_time','difficulty','nethash_scraped']] = coinwarz.loc[coin]
check_missing(result)

62

In [255]:
result[:10]

,#,Market Cap,Price,tag,algorithm,block_time,difficulty,nethash_scraped
Name,,,,,,,,
bitcoin,1,"$98,046,608,781","$5,366.33",BTC,SHA-256,673.0000,1.655292e+13,1.056378e+20
ethereum,2,"$14,119,168,507",$128.24,ETH,Ethash,13.2958,2.161568e+15,1.625753e+14
bitcoincash,3,"$3,156,523,320",$172.20,BCH,SHA-256,600.0000,5.434658e+11,0.000000e+00
litecoin,4,"$2,281,325,629",$35.49,LTC,Scrypt,181.0000,5.152158e+06,1.222561e+14
bitcoinsv,5,"$2,197,405,214",$119.89,BSV,SHA-256,571.0000,3.741698e+11,2.814443e+18
cardano,6,"$700,197,528",$0.027006,NaN,NaN,NaN,NaN,NaN
tron,7,"$674,746,961",$0.010119,NaN,NaN,NaN,NaN,NaN
monero,8,"$656,752,466",$37.57,XMR,RandomX,117.0000,1.409307e+11,1.204536e+09
ethereumclassic,9,"$552,141,573",$4.75,ETC,Ethash,13.0065,1.603803e+14,1.233078e+13


In [256]:
found_data = result[~result.isnull().any(axis=1)].copy()
found_data

,#,Market Cap,Price,tag,algorithm,block_time,difficulty,nethash_scraped
Name,,,,,,,,
bitcoin,1,"$98,046,608,781","$5,366.33",BTC,SHA-256,673.0000,1.655292e+13,1.056378e+20
ethereum,2,"$14,119,168,507",$128.24,ETH,Ethash,13.2958,2.161568e+15,1.625753e+14
bitcoincash,3,"$3,156,523,320",$172.20,BCH,SHA-256,600.0000,5.434658e+11,0.000000e+00
litecoin,4,"$2,281,325,629",$35.49,LTC,Scrypt,181.0000,5.152158e+06,1.222561e+14
bitcoinsv,5,"$2,197,405,214",$119.89,BSV,SHA-256,571.0000,3.741698e+11,2.814443e+18
monero,8,"$656,752,466",$37.57,XMR,RandomX,117.0000,1.409307e+11,1.204536e+09
ethereumclassic,9,"$552,141,573",$4.75,ETC,Ethash,13.0065,1.603803e+14,1.233078e+13
dash,10,"$446,401,779",$47.59,DASH,X11,158.0000,1.427197e+08,3.879598e+15
zcash,11,"$243,247,669",$25.90,ZEC,Equihash,75.0000,3.864250e+07,4.220791e+09


In [257]:
found_data['algorithm'].value_counts()

Scrypt              7
SHA-256             7
Equihash            3
X11                 2
Ethash              2
RandomX             1
X16Rv2              1
Groestl             1
Lyra2REv2           1
MTP                 1
TimeTravel10        1
Blake (14r)         1
Lyra2REv3           1
BCD                 1
BeamHashII          1
Keccak              1
Equihash (210,9)    1
ZelHash             1
Zhash               1
CryptoNight         1
Ubqhash             1
CuckooCycle         1
Name: algorithm, dtype: int64

In [258]:
bitcoin_forks = ['bitcoin', 'bitcoinsv', 'bitcoincash','bitcoinsv','peercoin', 'litecoincash']

In [259]:
def difficulty_factor(coin):
    if coin.name in bitcoin_forks:
        return 2**32
    if coin.name == 'zcash':
        return 2**13
    if coin.name == 'litecoin':
        return 2**32
    if coin.name == 'ethereum' or coin.name == 'ethereumclassic':
        return 1
    else:
        return 0
    

In [261]:
def hardware_hashrate(coin): # Max hashes per second for the hardware
    if coin['algorithm'] == 'SHA-256':
        return 16*10**12
    if coin['algorithm'] == 'Scrypt':
        return 504*10**6
    if coin['algorithm'] == 'Ethash':
        return 190*10**6
    if coin['algorithm'] == 'Equihash':
        return 42*10**3
    return 0

In [260]:
# Energy use in Watts
def hardware_consumption(coin): 
    if coin['algorithm'] == 'SHA-256':
        return 1280
    if coin['algorithm'] == 'Scrypt':
        return 970
    if coin['algorithm'] == 'Ethash':
        return 800
    if coin['algorithm'] == 'Equihash':
        return 760
    return 0

In [235]:
def hardware_consumption_hash(coin): # Joule per hash
    if coin['algorithm'] == 'SHA-256':
        return 0.1/(10**9)
    if coin['algorithm'] == 'Scrypt':
        return 1.6/(10**6) # Antminer L3+ https://www.asicminervalue.com/miners/bitmain/antminer-l3-504mh
    if coin['algorithm'] == 'Ethash':
        return 4/(10**6) # Bitmain Antminer E3 – Ethereum 190 MH/S
    else: 
        return 0

In [262]:
found_data['difficulty_factor'] = found_data.apply(lambda x: difficulty_factor(x), axis=1)
found_data['hardware_hashrate'] = found_data.apply(lambda x: hardware_hashrate(x), axis=1)
found_data['hardware_consumption'] = found_data.apply(lambda x: hardware_consumption(x), axis=1)
found_data

,#,Market Cap,Price,tag,algorithm,block_time,difficulty,nethash_scraped,difficulty_factor,hardware_hashrate,hardware_consumption
Name,,,,,,,,,,,
bitcoin,1,"$98,046,608,781","$5,366.33",BTC,SHA-256,673.0000,1.655292e+13,1.056378e+20,4294967296,16000000000000,1280
ethereum,2,"$14,119,168,507",$128.24,ETH,Ethash,13.2958,2.161568e+15,1.625753e+14,1,190000000,800
bitcoincash,3,"$3,156,523,320",$172.20,BCH,SHA-256,600.0000,5.434658e+11,0.000000e+00,4294967296,16000000000000,1280
litecoin,4,"$2,281,325,629",$35.49,LTC,Scrypt,181.0000,5.152158e+06,1.222561e+14,4294967296,504000000,970
bitcoinsv,5,"$2,197,405,214",$119.89,BSV,SHA-256,571.0000,3.741698e+11,2.814443e+18,4294967296,16000000000000,1280
monero,8,"$656,752,466",$37.57,XMR,RandomX,117.0000,1.409307e+11,1.204536e+09,0,0,0
ethereumclassic,9,"$552,141,573",$4.75,ETC,Ethash,13.0065,1.603803e+14,1.233078e+13,1,190000000,800
dash,10,"$446,401,779",$47.59,DASH,X11,158.0000,1.427197e+08,3.879598e+15,0,0,0
zcash,11,"$243,247,669",$25.90,ZEC,Equihash,75.0000,3.864250e+07,4.220791e+09,8192,42000,760


In [267]:
found_data['nethash_calc'] = found_data['difficulty']*found_data['difficulty_factor']/found_data['block_time'] # amount of hashes per second
found_data['num_hardware'] = found_data['nethash_calc']/found_data['hardware_hashrate']
found_data['energy_per_second'] = found_data['num_hardware']*found_data['hardware_consumption'] # in Watt
found_data['annual_energy'] = found_data['energy_per_second']*8760 # hours in a year
found_data

,#,Market Cap,Price,tag,algorithm,block_time,difficulty,nethash_scraped,difficulty_factor,hardware_hashrate,hardware_consumption,nethash_calc,num_hardware,energy_per_second,annual_energy
Name,,,,,,,,,,,,,,,
bitcoin,1,"$98,046,608,781","$5,366.33",BTC,SHA-256,673.0000,1.655292e+13,1.056378e+20,4294967296,16000000000000,1280,1.056378e+20,6.602365e+06,8.451027e+09,7.403100e+13
ethereum,2,"$14,119,168,507",$128.24,ETH,Ethash,13.2958,2.161568e+15,1.625753e+14,1,190000000,800,1.625753e+14,8.556594e+05,6.845275e+08,5.996461e+12
bitcoincash,3,"$3,156,523,320",$172.20,BCH,SHA-256,600.0000,5.434658e+11,0.000000e+00,4294967296,16000000000000,1280,3.890279e+18,2.431425e+05,3.112224e+08,2.726308e+12
litecoin,4,"$2,281,325,629",$35.49,LTC,Scrypt,181.0000,5.152158e+06,1.222561e+14,4294967296,504000000,970,1.222561e+14,2.425716e+05,2.352944e+08,2.061179e+12
bitcoinsv,5,"$2,197,405,214",$119.89,BSV,SHA-256,571.0000,3.741698e+11,2.814443e+18,4294967296,16000000000000,1280,2.814443e+18,1.759027e+05,2.251554e+08,1.972362e+12
monero,8,"$656,752,466",$37.57,XMR,RandomX,117.0000,1.409307e+11,1.204536e+09,0,0,0,0.000000e+00,NaN,NaN,NaN
ethereumclassic,9,"$552,141,573",$4.75,ETC,Ethash,13.0065,1.603803e+14,1.233078e+13,1,190000000,800,1.233078e+13,6.489884e+04,5.191907e+07,4.548111e+11
dash,10,"$446,401,779",$47.59,DASH,X11,158.0000,1.427197e+08,3.879598e+15,0,0,0,0.000000e+00,NaN,NaN,NaN
zcash,11,"$243,247,669",$25.90,ZEC,Equihash,75.0000,3.864250e+07,4.220791e+09,8192,42000,760,4.220791e+09,1.004950e+05,7.637623e+07,6.690557e+11


In [276]:
total_energy_crypto = np.sum(found_data['annual_energy'])
total_energy_crypto_TWh = total_energy/10**12

In [277]:
total_energy_world_TWh = 19504 # TWh 

In [278]:
total_energy_crypto_TWh/total_energy_world_TWh

0.004507540199581062

0.4% of global energy